In [26]:
import vk_api
import sqlite3

In [27]:
access_token = ""

In [28]:
def read_positions(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT positions, name FROM data")
    professions_positions = [(row[1], position) for row in cursor.fetchall() for position in row[0].split(',')]
    conn.close()
    return professions_positions

In [29]:
def search_users_by_job(vk_session, job, count=1000):
    try:
        search_results = vk_session.method('users.search', {
            'company': job,
            'count': count
        })
        return search_results['items']
    except Exception as e:
        print(f"Ошибка при поиске: {e}")
        return []


In [30]:
def get_user_groups(vk_session, user_id):
    try:
        groups = vk_session.method('groups.get', {'user_id': user_id})
        return ','.join(map(str, groups['items']))
    except Exception as e:
        return ''

In [31]:
def insert_user_data(db_path, user_id, profession, group_ids):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO users (user_id, profession, group_ids) VALUES (?, ?, ?)", (user_id, profession, group_ids))
    conn.commit()
    conn.close()

In [32]:
db_professions_path = "../../data/professions.db"
db_users = "../../data/users.db"

In [33]:
def main():
    vk_session = vk_api.VkApi(token=access_token)
    db_professions_path = "../../data/professions.db"
    db_users = "../../data/users.db"
    professions = read_positions(db_professions_path)
    for job in professions:
        users = search_users_by_job(vk_session, job[1])
        for user in users:
            groups = get_user_groups(vk_session, user['id'])
            if groups != "":
                insert_user_data(db_users, user['id'], job[0], groups)

In [34]:
if __name__ == '__main__':
    main()